
Градиентный бустинг


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV
import numpy as np


Читаем данные 

In [ ]:
df = pd.read_csv('file.csv', sep = ';', index_col='ID')
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
ID,,,,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29998,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1


Разделяем предикторы и отклик

In [ ]:
X = df.iloc[:, :-1].values  #
y = df.iloc[:, 23].values  # default_payment_next_month
X_train, X_holdout, y_train, y_holdout = train_test_split(X,y, test_size=0.3,random_state=17)

Подбираем параметры при помощи GridSearchCV

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {  
    'n_estimators': [10, 20, 30 ], 
    'learning_rate' : [0.05, 0.1]
}
model = GradientBoostingClassifier(random_state=17)
grad_boost_res = GridSearchCV(model, param_grid=param_grid, cv=2, error_score='raise', 
                               pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

grad_boost_res.fit(X_train, y_train)

grad_boost_res.best_params_, grad_boost_res.best_score_

({'learning_rate': 0.1, 'n_estimators': 20}, 0.8193333333333334)

In [ ]:
#Тренируем модель на полученных параметрах
grad_boost_res.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
     

In [ ]:
# Строим предсказание модели
y_pred_train = grad_boost_res.predict(X_train)
y_pred_test = grad_boost_res.predict(X_holdout)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy on train set is ", accuracy_score(y_train, y_pred_train)*100)
print("Accuracy on test set is ", accuracy_score(y_holdout, y_pred_test)*100)

Accuracy on train set is  82.19047619047619
Accuracy on test set is  82.31111111111112


Посмотрим на вероятности и решим, нужна ли калибровка.

In [ ]:
print(grad_boost_res.predict_proba(X_train))
print(grad_boost_res.predict_proba(X_holdout))

[[0.84929117 0.15070883]
 [0.88434037 0.11565963]
 [0.37206021 0.62793979]
 ...
 [0.74411322 0.25588678]
 [0.77040899 0.22959101]
 [0.85733286 0.14266714]]
[[0.87589468 0.12410532]
 [0.85358288 0.14641712]
 [0.87589468 0.12410532]
 ...
 [0.7803971  0.2196029 ]
 [0.81724982 0.18275018]
 [0.87589468 0.12410532]]


Калибровка

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
model_sigmoid = CalibratedClassifierCV(grad_boost_res, cv=2, method='sigmoid')
model_sigmoid.fit(X_train, y_train)
accuracy_score(y_holdout, model_sigmoid.predict(X_holdout))

0.8234444444444444